In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [3]:
num_cities = 5

# Generate a symmetric distance matrix for TSP with random distances
# Using the upper triangle and then adding its transpose to get the full matrix
np.random.seed(0)  # Seed for reproducibility
upper_triangle = np.random.randint(1, 100, size=(num_cities, num_cities))
distance_matrix = np.triu(upper_triangle, k=1)  # Upper triangle without the diagonal
distance_matrix += distance_matrix.T  # Make the matrix symmetric

distance_matrix

array([[ 0, 48, 65, 68, 68],
       [48,  0, 22, 37, 88],
       [65, 22,  0, 13, 59],
       [68, 37, 13,  0, 89],
       [68, 88, 59, 89,  0]])

256

In [4]:
import math
import random

def calculate_total_distance(route, distance_matrix):
    total_dist = 0
    for i in range(len(route)):
        total_dist += distance_matrix[route[i-1]][route[i]]
    return total_dist

def swap_two_cities(route):
    new_route = route.copy()
    i, j = random.sample(range(len(route)), 2)
    new_route[i], new_route[j] = new_route[j], new_route[i]
    return new_route

def simulated_annealing(distance_matrix, initial_temp=100000, cooling_rate=0.00003, stopping_temp=1):
    current_temp = initial_temp
    current_route = random.sample(range(len(distance_matrix)), len(distance_matrix))
    current_distance = calculate_total_distance(current_route, distance_matrix)
    
    while current_temp > stopping_temp:
        new_route = swap_two_cities(current_route)
        new_distance = calculate_total_distance(new_route, distance_matrix)

        if new_distance < current_distance or random.uniform(0, 1) < math.exp((current_distance - new_distance) / current_temp):
            current_route = new_route
            current_distance = new_distance

        current_temp *= 1 - cooling_rate

    return current_route, current_distance

best_route, best_distance = simulated_annealing(distance_matrix)
print("最佳路径:", best_route)
print("路径长度:", best_distance)

最佳路径: [4, 2, 3, 1, 0]
路径长度: 225


In [8]:
# 定义TSP问题的距离矩阵
distance_matrix = np.array([
    [np.inf, 2, 9, 10],
    [1, np.inf, 6, 4],
    [15, 7, np.inf, 8],
    [6, 3, 12, np.inf]
])
big_number = 1e6
distance_matrix[distance_matrix == np.inf] = big_number
num_cities = distance_matrix.shape[0]

# 定义线性规划问题的目标函数系数
# 我们需要为每一对城市创建一个变量
c = distance_matrix.flatten()

# 定义初始约束 - 每个城市必须被离开一次和进入一次
# 约束矩阵
A_eq = np.zeros((2*num_cities, num_cities**2))
for i in range(num_cities):
    A_eq[i, i::num_cities] = 1  # 离开城市i的约束
    A_eq[i + num_cities, num_cities*i:num_cities*(i+1)] = 1  # 进入城市i的约束

# 约束值
b_eq = np.ones(2*num_cities)

# 变量界限，每个决策变量在0和1之间
bounds = [(0, 1) for _ in range(num_cities**2)]

# 解决初始线性规划问题
res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# 检查结果中是否有子循环，并添加约束来消除它们
# 这里没有展示这一步，因为它需要复杂的逻辑来识别子循环
# 并构建相应的约束来消除子循环

# 输出结果
print('Objective value:', res.fun)

# 输出线性规划松弛解
# 这通常不是TSP的有效解，但可以给出一个下界
print('LP relaxation solution:', res.x)

Objective value: 21.0
LP relaxation solution: [ 0. -0.  1.  0.  1.  0. -0. -0.  0.  0.  0.  1.  0.  1.  0.  0.]


In [10]:
A_eq

array([[1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
       [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.]])

In [7]:
import pulp
import itertools
import numpy as np

# 创建一个 TSP 问题实例
def create_tsp_problem(num_cities, distance_matrix):
    problem = pulp.LpProblem("TSP", pulp.LpMinimize)

    # 创建决策变量：如果城市i和j之间的路径被选择，则x[(i, j)] = 1
    x = pulp.LpVariable.dicts('x', itertools.product(range(num_cities), range(num_cities)), 
                              lowBound=0, upBound=1, cat=pulp.LpBinary)

    # 目标函数：最小化总距离
    problem += pulp.lpSum(distance_matrix[i][j] * x[(i, j)] for i in range(num_cities) for j in range(num_cities))

    # 约束：每个城市必须离开一次和进入一次
    for i in range(num_cities):
        problem += pulp.lpSum(x[(i, j)] for j in range(num_cities) if i != j) == 1
        problem += pulp.lpSum(x[(j, i)] for j in range(num_cities) if i != j) == 1

    return problem, x

# 检查并添加子循环排除约束
def add_subtour_constraints(problem, x, num_cities, solution):
    # 找到解决方案中的所有子循环
    subtours = find_subtours(solution, num_cities)
    for subtour in subtours:
        problem += pulp.lpSum(x[(i, j)] for i, j in itertools.permutations(subtour, 2)) <= len(subtour) - 1

# 查找解决方案中的所有子循环
def find_subtours(solution, num_cities):
    # 构建图
    graph = {i: [] for i in range(num_cities)}
    for (i, j), value in solution.items():
        if value == 1:  # 如果城市i到城市j有路径
            graph[i].append(j)

    visited = [False] * num_cities
    subtours = []

    def dfs(city, start, path):
        visited[city] = True
        path.append(city)
        for next_city in graph[city]:
            if not visited[next_city]:
                dfs(next_city, start, path)
            elif next_city == start and len(path) > 1:
                # 发现子循环
                subtours.append(path.copy())

    for i in range(num_cities):
        if not visited[i]:
            dfs(i, i, [])

    # 去除重复的子循环
    unique_subtours = []
    for subtour in subtours:
        if set(subtour) not in [set(s) for s in unique_subtours]:
            unique_subtours.append(subtour)

    return unique_subtours

# 创建 TSP 问题实例
num_cities = 5
# distance_matrix = np.random.rand(num_cities, num_cities) * 100
# np.fill_diagonal(distance_matrix, 0)  # 填充对角线为0，因为不需要自环
problem, x = create_tsp_problem(num_cities, distance_matrix)

# 初始求解
solution_found = False
while not solution_found:
    problem.solve()

    # 提取当前解
    solution = {(i, j): pulp.value(x[(i, j)]) for i in range(num_cities) for j in range(num_cities)}

    # 检查是否有子循环
    if not find_subtours(solution, num_cities):  # 如果没有子循环，则找到了有效的解
        solution_found = True
    else:
        # 添加子循环排除约束
        add_subtour_constraints(problem, x, num_cities, solution)

# 输出最优解
print("最优路径总距离:", pulp.value(problem.objective))
for i in range(num_cities):
    for j in range(num_cities):
        if pulp.value(x[(i, j)]) == 1:
            print(f"从城市 {i} 到城市 {j}")

KeyboardInterrupt: 

In [14]:
solution

{(0, 0): None,
 (0, 1): 0.5,
 (0, 2): 0.0,
 (0, 3): 0.5,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): None,
 (1, 2): 1.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): None,
 (2, 3): 0.5,
 (2, 4): 0.5,
 (3, 0): 0.5,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): None,
 (3, 4): 0.5,
 (4, 0): 0.5,
 (4, 1): 0.5,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): None}